# Generate Umich Course Dependency Graph

Scrape UMich Course Website and Visualize Course Dependencies

**Author:** Harris Zheng

**Date:** January 16th, 2024

### 0. Load Packages

In [1]:
import requests
import pprint
import json
import bs4
import re
from bs4 import BeautifulSoup
from typing import Generator, Iterator, Any, TypeAlias, TypedDict, Union, Literal
import pandas as pd
import networkx as nx

import numpy as np
from pyvis.network import Network
import matplotlib.pyplot as plt

In [2]:
Column : TypeAlias = str
Record : TypeAlias = dict[Column, Any]

class CourseRecord(TypedDict):
    course_name: str
    description: str
    precede_prerequisites: list[int]
    precede_or_accompanied_prerequisites: list[int]

In [3]:
# chrome_headers = None
# with open("user_agent_headers/chrome_incognito_header.json", mode="r") as f:
#     chrome_headers = json.load(f)
#     # del chrome_headers["Authority"]
#     # del chrome_headers["Cookie"]

### 1. Read HTML

In [4]:
html : str = None
with open("umich.html", mode="r") as f:
    html = f.read()

soup : bs4.BeautifulSoup = BeautifulSoup(html, 'html.parser')

### 2. Parse HTML and Store Contents in a List of Records

In [5]:
course_text_generator : Iterator[bs4.element.Tag] = soup.find(id="main-content").find(class_="body wysiwyg-content").children
first_h2_position = 0
for i, course_tag in enumerate(course_text_generator):
    if course_tag.name == "h2":
        first_h2_position = i
        break

course_text_generator : Iterator[bs4.element.Tag] = soup.find(id="main-content").find(class_="body wysiwyg-content").children

In [6]:
list_of_records : list[CourseRecord] = []
new_record = {key: None for key in CourseRecord.__required_keys__}
num_steps_since_new_record = 0

for i, course_tag in enumerate(course_text_generator):
    if course_tag.text == "\n":
        continue

    if course_tag.name == "h2":
        if i != first_h2_position:
            list_of_records.append(new_record)
        new_record = {key: None for key in CourseRecord.__required_keys__}## Create new record (new course) on every encounter of h2
        num_steps_since_new_record = 0
        new_record["course_name"] = course_tag.text
        new_record["course_number"] = int(re.search("[0-9]+", course_tag.text).group())
        
    else:
        num_steps_since_new_record += 1

        # First tag after h2 is always the description tag
        if num_steps_since_new_record == 1:
            new_record["description"] = course_tag.text
        else:
            tag_text = course_tag.text
            if "Prerequisite" not in tag_text:
                continue
        
            # precede_start_position = re.search("(preceded (?!or accompanied))*.*", flags=re.IGNORECASE).start() # negative lookahead expression
            precede_or_accompanied_match : re.Match = re.search("(preceded or accompanied)+", tag_text, flags=re.IGNORECASE)
            
            if precede_or_accompanied_match:
                precede_or_accompanied_position = precede_or_accompanied_match.start()    
                precede_or_accompanied_text = tag_text[precede_or_accompanied_position:]
                precede_or_accompanied_prerequisites = [int(number) for number in re.findall("[0-9]+", precede_or_accompanied_text)]
                new_record["precede_or_accompanied_prerequisites"] = (
                    precede_or_accompanied_prerequisites 
                    if len(precede_or_accompanied_prerequisites) != 0
                    else None
                ) 
                
                precede_text = tag_text[:precede_or_accompanied_position]
                precede_prerequisites = [int(number) for number in re.findall("[0-9]+", precede_text)]
                new_record["precede_prerequisites"] = (
                    precede_prerequisites
                    if len(precede_prerequisites) != 0
                    else None
                )   
            else:
                precede_prerequisites = [int(number) for number in re.findall("[0-9]+", tag_text)]
                new_record["precede_prerequisites"] = (
                    precede_prerequisites
                    if len(precede_prerequisites) != 0
                    else None
                )   

list_of_records.append(new_record)
                   

In [7]:
df_umich_courses : pd.DataFrame = pd.DataFrame.from_records(list_of_records)

In [8]:
df_umich_courses.columns

Index(['course_name', 'precede_or_accompanied_prerequisites', 'description',
       'precede_prerequisites', 'course_number'],
      dtype='object')

In [9]:
# Reorder columns
df_umich_courses : pd.DataFrame = (
    df_umich_courses.iloc[
        :, 
        df_umich_courses
            .columns
            .reindex(
                [
                    'course_number', 
                    'course_name', 
                    'description', 
                    'precede_prerequisites', 
                    'precede_or_accompanied_prerequisites'
                ])
            [1]
    ]
)

In [10]:
df_umich_courses.sort_values(by="course_number", inplace=True)

In [11]:
df_umich_courses.head()

,course_number,course_name,description,precede_prerequisites,precede_or_accompanied_prerequisites
0,501,SIADS 501 - Being a Data Scientist,"This course explores what expertise, perspecti...",None,None
1,502,SIADS 502 - Math Methods I,This course covers foundational topics in line...,None,None
2,503,SIADS 503 - Data Science Ethics,The course introduces the ethical challenges t...,[501],None
3,505,SIADS 505 - Data Manipulation,This course will introduce data manipulation a...,None,None
4,511,SIADS 511 - SQL and Databases,This course will introduce the students to beg...,None,None


### 3. Create a Course Completion DataFrame and Write Out to csv for User Input  (Run Only Once)

In [ ]:

df_umich_courses_completion = df_umich_courses[["course_number", "course_name"]].copy()

In [ ]:

df_umich_courses_completion["status"] = "incomplete"

In [ ]:

df_umich_courses_completion.to_csv("course_completion.csv", index=False)

### 4. Read Course Completion CSV and Prepare Adjacency Matrix to Construct/Optimize Graph Nodes and Edges

#### 4.1 Course Completion CSV

In [12]:
df_umich_courses_completion = pd.read_csv("course_completion.csv")

#### 4.2 Adjacency List Seperated by PrerequisiteType (Either Precede, Precede or Accompanied)

In [13]:
PrerequisiteType: TypeAlias = Literal["precede_prerequisites", "precede_or_accompanied_prerequisites"]
CourseNode : TypeAlias = int # represents course number of node
CourseStatus: TypeAlias = str
CoursePrequisites : TypeAlias = list[int] # represents prerequisites of course node 

class UmichCourseGraphByType(TypedDict):
    CourseNodeInfo : dict[PrerequisiteType, dict[CourseNode, CoursePrequisites]]        

In [14]:
graph_umich_courses : UmichCourseGraphByType = (
    df_umich_courses[["course_number", "precede_prerequisites", "precede_or_accompanied_prerequisites"]]
        .set_index("course_number")
        .to_dict()
)

#### 4.3 Adjacency List Not Separated by Prerequisite Type (Either Precede, Precede or Accompanied)

In [15]:
class UmichCourseGraph(TypedDict):
    CourseNodeInfo : dict[CourseNode, CoursePrequisites]

In [16]:
df_umich_courses_full_prerequisites : pd.DataFrame = df_umich_courses.copy()

In [17]:
def combine_prerequisites(row):
    all_prereq_array = []
    if isinstance(row["precede_prerequisites"], list):
        all_prereq_array.extend(row["precede_prerequisites"])
    
    if isinstance(row["precede_or_accompanied_prerequisites"], list):
        all_prereq_array.extend(row["precede_or_accompanied_prerequisites"])

    return all_prereq_array

In [18]:
df_umich_courses_full_prerequisites["all_prerequisites"] = df_umich_courses_full_prerequisites.apply(combine_prerequisites, axis=1)

In [19]:
df_adjacency_list : UmichCourseGraph = (
    df_umich_courses_full_prerequisites[["course_number", "all_prerequisites"]]
        .set_index("course_number")
        .to_dict()['all_prerequisites']
)

#### 4.3.1 Check Adjacency List Construction

I want to quickly check that we are not missing any prerequisites in combine_prerequisites

From Eye Level:

In [20]:
df_umich_courses_full_prerequisites.set_index('course_number')['all_prerequisites'].head(10)

course_number
501            []
502            []
503         [501]
505            []
511            []
515            []
516    [511, 505]
521         [505]
522         [505]
523         [522]
Name: all_prerequisites, dtype: object

From Asserts:

In [21]:
both_accompanied_and_preceded = df_umich_courses.loc[
    (~df_umich_courses["precede_prerequisites"].isnull()) & 
    (~df_umich_courses["precede_or_accompanied_prerequisites"].isnull())
].set_index("course_number")

b = pd.DataFrame(both_accompanied_and_preceded["precede_prerequisites"] + both_accompanied_and_preceded["precede_or_accompanied_prerequisites"], columns=["all_prerequisites"])
b

,all_prerequisites
course_number,
643,"[502, 505, 532, 542, 602, 511]"
644,"[502, 505, 532, 542, 602, 642, 632]"
655,"[502, 505, 532, 542, 602, 515, 632]"
682,"[501, 502, 503, 505, 515, 521, 532, 542, 602, ..."
688,"[501, 503, 505, 511, 515, 516, 521, 522, 593, ..."
699,"[593, 502, 524, 532, 542, 543, 602, 630, 632, ..."


In [22]:
a = df_umich_courses_full_prerequisites.set_index('course_number')[['all_prerequisites']]
a

,all_prerequisites
course_number,
501,[]
502,[]
503,[501]
505,[]
511,[]
515,[]
516,"[511, 505]"
521,[505]
522,[505]


In [23]:
prerequisites_a_b = pd.merge(a, b, left_index=True, right_index=True)

In [24]:
assert (prerequisites_a_b["all_prerequisites_x"] == prerequisites_a_b["all_prerequisites_y"]).all(), "Not all Prerequisites Equal"

### 5 Now with All Assets Prepared, We Build Functions which Constructs and Simplifies our Depdency Graph!

In [25]:
import random
random.seed(10)

In [29]:
def initialize_node_positions(nodes : tuple[CourseNode, 
                                            CourseStatus,
                                            CoursePrequisites, 
                                            CoursePrequisites], 
                             shape : tuple[int, int]) -> nx.DiGraph:
    nt : nx.DiGraph() = nx.DiGraph()
    
    for i, (course_node, course_status) in enumerate(nodes):
        nt.add_node(course_node, 
                    size=20, 
                    label=str(course_node), 
                    shape="circle",
                    status=course_status,
                    subset=0 if course_status in ("complete", "in progress") else random.randint(1,5),
                    physics=False,
        )

    pos = nx.multipartite_layout(nt, scale=300)
    for node, node_attr in nt.nodes.items():
        nt.nodes[node]["x"] = pos[node][0] - (500 * 0.5 if node_attr["status"] in ("complete", "in progress") else 0)
        nt.nodes[node]["y"] = pos[node][1]

    return nt

In [39]:
def toNetwork(graph: UmichCourseGraphByType, nt: nx.DiGraph)->  nx.DiGraph:
    def checkKeyExists(name):
        # if node does not exist, continue operations without that node
        return name in nt
        # if name not in nt:
            # nt.add_node(name, size=40, label=str(name), shape="circle")

    for i, (type_prerequisite, course_nodes) in enumerate(graph.items()):
            for node in course_nodes:
                if (not checkKeyExists(node)) or course_nodes[node] is None:
                    continue
                for child in course_nodes[node]:
                    if (not checkKeyExists(child)):
                        continue
                    if type_prerequisite == "precede_prerequisites":
                        nt.add_edge(child,node, prerequisite_type=type_prerequisite, color="blue", physics=False,
                                    title=type_prerequisite)
                    else:
                        nt.add_edge(child,node, prerequisite_type=type_prerequisite, color="red", physics=False,
                                    title=type_prerequisite)
    return nt

def ntw_pyvis(ntx:nx.DiGraph):
    net = Network(width="100%",height="800px", directed=True, notebook=True,
                  cdn_resources='remote', select_menu=False, filter_menu=False)
    net.from_nx(ntx)
    return net

In [40]:
G = initialize_node_positions(df_umich_courses_completion[["course_number", "status"]].to_records(index=False).tolist(), (5,8))
network = toNetwork(graph_umich_courses, G)

In [41]:
# network.remove_node(22)

In [42]:
print("Number of Nodes:", len(network.nodes))
print("Number of Edges:", len(network.edges))

Number of Nodes: 37
Number of Edges: 151


In [43]:
assert len(network.nodes) == len(df_umich_courses)

In [44]:
net = ntw_pyvis(network)

In [45]:
net.get_node(503)

{'color': '#97c2fc',
 'size': 20,
 'status': 'complete',
 'subset': 0,
 'physics': False,
 'x': -440.5405405405405,
 'y': 75.0,
 'id': 503,
 'label': '503',
 'shape': 'circle'}

In [46]:
net.show_buttons(filter_="physics")

In [47]:
_ = net.show(name="umich_course_graph.html")

umich_course_graph.html
